In [1]:
import sys
import matplotlib
matplotlib.use('Agg') # Need this to write images from the GSA servers.  Order matters:
import matplotlib.pyplot as plt # First import matplotlib, then use Agg, then import plt
%matplotlib inline

# ML4CVD Imports
from ml4cvd.arguments import parse_args
from ml4cvd.tensor_generators import test_train_valid_tensor_generators
from ml4cvd.models import make_multimodal_to_multilabel_model, train_model_from_generators

/usr/local/lib/python3.6/dist-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '
Using TensorFlow backend.


In [2]:
sys.argv = ['train', 
            '--tensors','/mnt/disks/ecg-bike-tensors/2019-09-04/', 
            '--input_tensors', 'ecg-bike-trend-hr', 'ecg-bike-trend-load',
            '--output_tensors','ecg-bike-age', 'genetic_sex', 'bmi',
            '--batch_size', '64', 
            '--epochs', '200',
            '--training_steps', '5',
            '--validation_steps', '2',
            '--inspect_model',
            '--u_connect',
            '--id', 'ecg_new_output_tab',
            '--learning_rate', '1e-3',
           ]
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(args.tensor_maps_in,  args.tensor_maps_out,  args.tensors, args.batch_size,   args.valid_ratio, args.test_ratio, args.test_modulo, args.balance_csvs)

2019-09-06 17:21:06,625 - logger:25 - INFO - Logging configuration was loaded. Log messages can be found at ./recipes_output/ecg_new_output/log_2019-09-06_17-21_0.log.
2019-09-06 17:21:06,626 - arguments:180 - INFO - Command Line was:

python train --tensors /mnt/disks/ecg-bike-tensors/2019-09-04/ --input_tensors ecg-bike-trend-hr ecg-bike-trend-load --output_tensors ecg-bike-age genetic_sex bmi --batch_size 64 --epochs 200 --training_steps 5 --validation_steps 2 --inspect_model --u_connect --id ecg_new_output --learning_rate 1e-3


2019-09-06 17:21:06,627 - arguments:181 - INFO - Total TensorMaps:4459 Arguments are Namespace(activation='relu', aligned_dimension=16, alpha=0.5, app_csv=None, b_slice_force=None, balance_csvs=[], batch_size=64, bigquery_credentials_file='/mnt/ml4cvd/projects/jamesp/bigquery/bigquery-viewer-credentials.json', bigquery_dataset='broad-ml4cvd.ukbb7089_r10data', block_size=3, categorical_field_ids=[], continuous_field_ids=[], conv_bn=False, conv_dropout=0.0, c

In [3]:
model = make_multimodal_to_multilabel_model(args.model_file, args.model_layers, args.model_freeze,
                                                args.tensor_maps_in, args.tensor_maps_out, args.activation,
                                                args.dense_layers, args.dropout, args.mlp_concat, args.conv_layers,
                                                args.max_pools, args.res_layers, args.dense_blocks, args.block_size,
                                                args.conv_bn, args.conv_x, args.conv_y, args.conv_z, args.conv_dropout,
                                                args.conv_width, args.u_connect, args.pool_x, args.pool_y, args.pool_z, args.padding,
                                                args.learning_rate, optimizer='RAdam')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_trend_heartrate_ecg_bike  (None, 106, 1)       0                                            
__________________________________________________________________________________________________
input_trend_load_ecg_bike (Inpu (None, 106, 1)       0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 106, 32)      2304        input_trend_heartrate_ecg_bike[0]
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 106, 32)      2304        input_trend_load_ecg_bike[0][0]  
__________________________________________________________________________________________________
conv1d_2 (

In [4]:
model = train_model_from_generators(model, generate_train, generate_valid, args.training_steps,
                                    args.validation_steps, args.batch_size, args.epochs, args.patience,
                                    args.output_folder, args.id, args.inspect_model, args.inspect_show_labels)

2019-09-06 17:21:08,653 - models:1140 - INFO - Saving architecture diagram to:./recipes_output/ecg_new_output/architecture_graph_ecg_new_output.png
Epoch 1/1
2019-09-06 17:21:16,872 - tensor_generators:372 - INFO - At tensor path: /mnt/disks/ecg-bike-tensors/2019-09-04/2958609.hd5
2019-09-06 17:21:16,975 - tensor_generators:373 - INFO - Got first error: ValueError while attempting to generate tensor:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/tensor_generators.py", line 86, in multimodal_multitask_generator
    out_batch[tm.output_name()][stats['batch_index']] = tm.tensor_from_file(tm, hd5)
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/TensorMap.py", line 728, in _default_tensor_from_file
    raise ValueError(tm.name + ' is a continuous value that cannot be set to 0, but no value was found.')
ValueError: 23104_Body-mass-index-BMI_0_0 is a continuous value that cannot be set to 0, but no value was found.


2019-09-06 17:21:40,404 - t

5/5 [==============================] - 20s 4s/step - loss: 1.1666 - output_age_ecg_bike_loss: 0.2084 - output_genetic_sex_categorical_loss: 0.6407 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3175 - output_age_ecg_bike_logcosh: 0.2230 - output_genetic_sex_categorical_categorical_accuracy: 0.7250 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.7090 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6903 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.7272 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.7273 - val_loss: 1.1383 - val_output_age_ecg_bike_loss: 0.1885 - val_output_genetic_sex_categorical_loss: 0.6517 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2981 - val_output_age_ecg_bike_logcosh: 0.2025 - val_output_genetic_sex_categorical_categorical_accuracy: 0.6562 - val_output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.6538 - val_output_genetic_sex_categorical_Geneti


Epoch 00015: val_loss improved from 1.06863 to 0.89288, saving model to ./recipes_output/ecg_new_output/ecg_new_output.hd5
Epoch 16/200
5/5 [==============================] - 27s 5s/step - loss: 1.0688 - output_age_ecg_bike_loss: 0.1756 - output_genetic_sex_categorical_loss: 0.5686 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3246 - output_age_ecg_bike_logcosh: 0.1925 - output_genetic_sex_categorical_categorical_accuracy: 0.7156 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.7443 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6708 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.6739 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.7764 - val_loss: 1.0081 - val_output_age_ecg_bike_loss: 0.1983 - val_output_genetic_sex_categorical_loss: 0.4618 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3479 - val_output_age_ecg_bike_logcosh: 0.2011 - val_output_genetic_sex_categorical_categorical_a


Epoch 00028: val_loss did not improve from 0.89288
Epoch 29/200
5/5 [==============================] - 35s 7s/step - loss: 1.0825 - output_age_ecg_bike_loss: 0.1636 - output_genetic_sex_categorical_loss: 0.5942 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3246 - output_age_ecg_bike_logcosh: 0.1735 - output_genetic_sex_categorical_categorical_accuracy: 0.6906 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.6970 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6641 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.6724 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.7212 - val_loss: 0.9354 - val_output_age_ecg_bike_loss: 0.1930 - val_output_genetic_sex_categorical_loss: 0.5235 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2190 - val_output_age_ecg_bike_logcosh: 0.2067 - val_output_genetic_sex_categorical_categorical_accuracy: 0.7812 - val_output_genetic_sex_categorical_Genetic_sex_Female_


Epoch 00034: val_loss did not improve from 0.89288
Epoch 35/200
5/5 [==============================] - 34s 7s/step - loss: 1.0395 - output_age_ecg_bike_loss: 0.1659 - output_genetic_sex_categorical_loss: 0.5657 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3079 - output_age_ecg_bike_logcosh: 0.1805 - output_genetic_sex_categorical_categorical_accuracy: 0.7156 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.7464 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6343 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.7301 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.6991 - val_loss: 1.0824 - val_output_age_ecg_bike_loss: 0.1667 - val_output_genetic_sex_categorical_loss: 0.5832 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3325 - val_output_age_ecg_bike_logcosh: 0.1771 - val_output_genetic_sex_categorical_categorical_accuracy: 0.7344 - val_output_genetic_sex_categorical_Genetic_sex_Female_

In [10]:
model.save('./recipes_output/ecg_new_output_tab/ecg_new_output_tab.hd5')

# Raw data

In [5]:
sys.argv = ['train', 
            '--tensors','/mnt/disks/ecg-bike-tensors/2019-09-04/', 
            '--input_tensors', 'ecg-bike-trend-load', 'ecg-bike-recovery',
            '--output_tensors','ecg-bike-age', 'genetic_sex', 'bmi',
            '--batch_size', '64', 
            '--epochs', '200',
            '--training_steps', '5',
            '--validation_steps', '2',
            '--inspect_model',
            '--u_connect',
            '--id', 'ecg_new_output',
            '--learning_rate', '1e-3',
           ]
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(args.tensor_maps_in,  args.tensor_maps_out,  args.tensors, args.batch_size,   args.valid_ratio, args.test_ratio, args.test_modulo, args.balance_csvs)

2019-09-06 17:44:30,045 - logger:25 - INFO - Logging configuration was loaded. Log messages can be found at ./recipes_output/ecg_new_output/log_2019-09-06_17-44_0.log.
2019-09-06 17:44:30,089 - arguments:180 - INFO - Command Line was:

python train --tensors /mnt/disks/ecg-bike-tensors/2019-09-04/ --input_tensors ecg-bike-trend-load ecg-bike-recovery --output_tensors ecg-bike-age genetic_sex bmi --batch_size 64 --epochs 200 --training_steps 5 --validation_steps 2 --inspect_model --u_connect --id ecg_new_output --learning_rate 1e-3


2019-09-06 17:44:30,096 - arguments:181 - INFO - Total TensorMaps:4459 Arguments are Namespace(activation='relu', aligned_dimension=16, alpha=0.5, app_csv=None, b_slice_force=None, balance_csvs=[], batch_size=64, bigquery_credentials_file='/mnt/ml4cvd/projects/jamesp/bigquery/bigquery-viewer-credentials.json', bigquery_dataset='broad-ml4cvd.ukbb7089_r10data', block_size=3, categorical_field_ids=[], continuous_field_ids=[], conv_bn=False, conv_dropout=0.0, c

In [6]:
raw_model = make_multimodal_to_multilabel_model(args.model_file, args.model_layers, args.model_freeze,
                                                args.tensor_maps_in, args.tensor_maps_out, args.activation,
                                                args.dense_layers, args.dropout, args.mlp_concat, args.conv_layers,
                                                args.max_pools, args.res_layers, args.dense_blocks, args.block_size,
                                                args.conv_bn, args.conv_x, args.conv_y, args.conv_z, args.conv_dropout,
                                                args.conv_width, args.u_connect, args.pool_x, args.pool_y, args.pool_z, args.padding,
                                                args.learning_rate, optimizer='RAdam')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_trend_load_ecg_bike (Inpu (None, 106, 1)       0                                            
__________________________________________________________________________________________________
input_full_ecg_bike (InputLayer (None, 30000, 1)     0                                            
__________________________________________________________________________________________________
conv1d_27 (Conv1D)              (None, 106, 32)      2304        input_trend_load_ecg_bike[0][0]  
__________________________________________________________________________________________________
conv1d_40 (Conv1D)              (None, 30000, 32)    2304        input_full_ecg_bike[0][0]        
__________________________________________________________________________________________________
conv1d_28 

In [7]:
raw_model = train_model_from_generators(raw_model, generate_train, generate_valid, args.training_steps,
                                    args.validation_steps, args.batch_size, args.epochs, args.patience,
                                    args.output_folder, args.id, args.inspect_model, args.inspect_show_labels)

2019-09-06 17:44:35,666 - models:1140 - INFO - Saving architecture diagram to:./recipes_output/ecg_new_output/architecture_graph_ecg_new_output.png
Epoch 1/1
2019-09-06 17:44:41,277 - tensor_generators:372 - INFO - At tensor path: /mnt/disks/ecg-bike-tensors/2019-09-04/5677903.hd5
2019-09-06 17:44:41,279 - tensor_generators:373 - INFO - Got first error: ValueError while attempting to generate tensor:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/tensor_generators.py", line 80, in multimodal_multitask_generator
    in_batch[tm.input_name()][stats['batch_index']] = tm.tensor_from_file(tm, hd5, dependents)
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/tensor_from_file.py", line 58, in first_date_bike_recovery
    original = get_tensor_at_first_date(hd5, tm.group, DataSetType.FLOAT_ARRAY, tm.name)
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/tensor_from_file.py", line 36, in get_tensor_at_first_date
    first_date_path = min(all_d


Epoch 00002: val_loss improved from 1.44942 to 1.28373, saving model to ./recipes_output/ecg_new_output/ecg_new_output.hd5
Epoch 3/200
5/5 [==============================] - 74s 15s/step - loss: 1.2866 - output_age_ecg_bike_loss: 0.2759 - output_genetic_sex_categorical_loss: 0.6746 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3362 - output_age_ecg_bike_logcosh: 0.2885 - output_genetic_sex_categorical_categorical_accuracy: 0.5125 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.5651 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.4708 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.4880 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.5685 - val_loss: 1.2319 - val_output_age_ecg_bike_loss: 0.2097 - val_output_genetic_sex_categorical_loss: 0.6823 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3399 - val_output_age_ecg_bike_logcosh: 0.2240 - val_output_genetic_sex_categorical_categorical_a

5/5 [==============================] - 56s 11s/step - loss: 1.1294 - output_age_ecg_bike_loss: 0.1831 - output_genetic_sex_categorical_loss: 0.6423 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3040 - output_age_ecg_bike_logcosh: 0.2000 - output_genetic_sex_categorical_categorical_accuracy: 0.7000 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.7263 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6447 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.6764 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.7445 - val_loss: 1.2233 - val_output_age_ecg_bike_loss: 0.2207 - val_output_genetic_sex_categorical_loss: 0.6102 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3924 - val_output_age_ecg_bike_logcosh: 0.2530 - val_output_genetic_sex_categorical_categorical_accuracy: 0.7266 - val_output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.7391 - val_output_genetic_sex_categorical_Genet


Epoch 00020: val_loss did not improve from 0.97851
Epoch 21/200
5/5 [==============================] - 55s 11s/step - loss: 1.0671 - output_age_ecg_bike_loss: 0.1897 - output_genetic_sex_categorical_loss: 0.5488 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3287 - output_age_ecg_bike_logcosh: 0.2048 - output_genetic_sex_categorical_categorical_accuracy: 0.7250 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.7274 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6790 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.7078 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.7643 - val_loss: 1.0865 - val_output_age_ecg_bike_loss: 0.2322 - val_output_genetic_sex_categorical_loss: 0.6017 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2526 - val_output_age_ecg_bike_logcosh: 0.2400 - val_output_genetic_sex_categorical_categorical_accuracy: 0.6172 - val_output_genetic_sex_categorical_Genetic_sex_Female

2019-09-06 18:15:09,058 - tensor_generators:114 - INFO - ValueError while attempting to generate tensor:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/tensor_generators.py", line 80, in multimodal_multitask_generator
    in_batch[tm.input_name()][stats['batch_index']] = tm.tensor_from_file(tm, hd5, dependents)
  File "/usr/local/lib/python3.6/dist-packages/ml4cvd/tensor_from_file.py", line 53, in float_array_zero_pad_from_file_first_date
    padded[:original.shape[0]] = original.reshape((original.shape[0],) + tm.shape[1:])
ValueError: could not broadcast input array from shape (107,1) into shape (106,1)

: 2
2019-09-06 18:15:09,059 - tensor_generators:114 - INFO - epochs: 1
2019-09-06 18:15:09,060 - tensor_generators:115 - INFO - Generator looped & shuffled over 6584 tensors.
2019-09-06 18:15:09,061 - tensor_generators:116 - INFO - True epoch number:1 in which 5078 tensors were presented.
5/5 [==============================] - 54s 11s/step - l


Epoch 00035: val_loss did not improve from 0.95118
Epoch 36/200
5/5 [==============================] - 55s 11s/step - loss: 1.0556 - output_age_ecg_bike_loss: 0.1877 - output_genetic_sex_categorical_loss: 0.5526 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.3153 - output_age_ecg_bike_logcosh: 0.1993 - output_genetic_sex_categorical_categorical_accuracy: 0.7313 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.7214 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6998 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.7249 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.7502 - val_loss: 0.9135 - val_output_age_ecg_bike_loss: 0.1473 - val_output_genetic_sex_categorical_loss: 0.5370 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2293 - val_output_age_ecg_bike_logcosh: 0.1649 - val_output_genetic_sex_categorical_categorical_accuracy: 0.7422 - val_output_genetic_sex_categorical_Genetic_sex_Female


Epoch 00048: val_loss did not improve from 0.85239
Epoch 49/200
5/5 [==============================] - 57s 11s/step - loss: 0.9700 - output_age_ecg_bike_loss: 0.1738 - output_genetic_sex_categorical_loss: 0.5394 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2568 - output_age_ecg_bike_logcosh: 0.1815 - output_genetic_sex_categorical_categorical_accuracy: 0.7156 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.7527 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6418 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.7200 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.7317 - val_loss: 1.0114 - val_output_age_ecg_bike_loss: 0.1242 - val_output_genetic_sex_categorical_loss: 0.5898 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2974 - val_output_age_ecg_bike_logcosh: 0.1275 - val_output_genetic_sex_categorical_categorical_accuracy: 0.6094 - val_output_genetic_sex_categorical_Genetic_sex_Female


Epoch 00060: val_loss did not improve from 0.80702
Epoch 61/200
5/5 [==============================] - 55s 11s/step - loss: 0.9735 - output_age_ecg_bike_loss: 0.1583 - output_genetic_sex_categorical_loss: 0.5450 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2703 - output_age_ecg_bike_logcosh: 0.1691 - output_genetic_sex_categorical_categorical_accuracy: 0.7188 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.7991 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6517 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.6704 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.8206 - val_loss: 1.1346 - val_output_age_ecg_bike_loss: 0.1469 - val_output_genetic_sex_categorical_loss: 0.7066 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2812 - val_output_age_ecg_bike_logcosh: 0.1546 - val_output_genetic_sex_categorical_categorical_accuracy: 0.5781 - val_output_genetic_sex_categorical_Genetic_sex_Female


Epoch 00070: val_loss did not improve from 0.80702
Epoch 71/200
5/5 [==============================] - 56s 11s/step - loss: 0.9708 - output_age_ecg_bike_loss: 0.1580 - output_genetic_sex_categorical_loss: 0.5317 - output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2811 - output_age_ecg_bike_logcosh: 0.1651 - output_genetic_sex_categorical_categorical_accuracy: 0.7031 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_precision: 0.6963 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_precision: 0.6930 - output_genetic_sex_categorical_Genetic_sex_Female_0_0_recall: 0.6832 - output_genetic_sex_categorical_Genetic_sex_Male_0_0_recall: 0.7380 - val_loss: 0.9388 - val_output_age_ecg_bike_loss: 0.1484 - val_output_genetic_sex_categorical_loss: 0.5702 - val_output_23104_Body-mass-index-BMI_0_0_continuous_loss: 0.2201 - val_output_age_ecg_bike_logcosh: 0.1513 - val_output_genetic_sex_categorical_categorical_accuracy: 0.6875 - val_output_genetic_sex_categorical_Genetic_sex_Female